# Monitor ML runs live 

## Introduction

This guide will show you how to:

* Monitor training and evaluation metrics and losses live
* Monitor hardware resources during training

By the end of it, you will monitor your metrics, losses, and hardware live in Neptune!

## Setup

Install dependencies

In [ ]:
! pip install --quiet neptune-client==0.5.3 tensorflow==2.3.1

## Step 1: Create a basic training script

As an example I'll use a script that trains a Keras model on mnist dataset.

In [ ]:
from tensorflow import keras

# parameters
PARAMS = {'epoch_nr': 10,
          'batch_size': 256,
          'lr': 0.005,
          'momentum': 0.4,
          'use_nesterov': True,
          'unit_nr': 256,
          'dropout': 0.05}

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(PARAMS['unit_nr'], activation=keras.activations.relu),
    keras.layers.Dropout(PARAMS['dropout']),
    keras.layers.Dense(10, activation=keras.activations.softmax)
])

optimizer = keras.optimizers.SGD(lr=PARAMS['lr'],
                                 momentum=PARAMS['momentum'],
                                 nesterov=PARAMS['use_nesterov'], )

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Step 2: Initialize Neptune and create new run

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project='common/quickstarts',
                   api_token='ANONYMOUS')

Click on the link above to open this run in Neptune.

For now it is empty but keep the tab with run open to see what happens next. 

**Few explanations**

In the above code You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/colab-test-run' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project_qualified_name='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

## Step 3: Add logging for metrics and losses

Since we are using Keras we'll create a Callback that logs metrics and losses after every epoch. 

In [ ]:
class NeptuneLogger(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            run['batch/{}'.format(log_name)].log(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            run['epoch/{}'.format(log_name)].log(log_value)

We need to pass it to the `callbacks` argument. 

In [ ]:
model.fit(x_train, y_train,
          epochs=PARAMS['epoch_nr'],
          batch_size=PARAMS['batch_size'],
          callbacks=[NeptuneLogger()])

## Step 4: See results live in the UI

Go to the `Charts` section of the Neptune UI to see them. 

![image](https://neptune.ai/wp-content/uploads/docs-ml-runs-charts.png)

Neptune automatically logs the hardware consumption during the run. 

You can see it in the `Monitoring` section of the Neptune UI. 

![image](https://neptune.ai/wp-content/uploads/docs-ml-runs-hardware.png)